In [1]:
import pandas as pd
import pickle
import numpy as np
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

from patsy import dmatrices

%matplotlib inline


In [3]:
df = pickle.load(open('df_dsmpl_orginal_charged_50.p', 'rb'))

In [24]:
df.columns

Index(['id', 'name', 'date', 'manner_of_death', 'armed', 'age', 'gender',
       'race', 'city', 'state', 'signs_of_mental_illness', 'threat_level',
       'flee', 'body_camera', 'age_right', 'armed_right', 'blurb',
       'description', 'editor_note', 'is_body_camera', 'is_geocoding_exact',
       'is_officer_charged', 'lat', 'lon', 'mental', 'name_right', 'photos',
       'race_right', 'sources', 'state_right', 'threat_level_display',
       'videos', 'weapon', 'charged_bool'],
      dtype='object')

In [10]:
df_income = pickle.load(open('df_income_save.p', 'rb'))

# pickle.dump(df_police, open( "df_police_save.p", "wb" ) )
# pickle.dump(df_income, open( "df_income_save.p", "wb" ) )
# pickle.dump(df_race, open( "df_race_save.p", "wb" ) )
# pickle.dump(df_latlon, open( "df_latlon_save.p", "wb" ) )


### Merge Zip-City and Zip-Mean CSV


In [15]:
df_income.head(2)

,Zip,Median,Mean,Pop
0,1001,"56,663","66,688","16,445"
1,1002,"49,853","75,063","28,069"


In [13]:
df_zip_city = pd.read_csv('./zipcodes-by-city.csv')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
#df_zip_city.head(2)
df_zip_city_reduced = df_zip_city[['Zipcode','City','State','TotalWages','EstimatedPopulation' ]]

In [21]:
df_zip_city_reduced.head(2)

,Zipcode,City,State,TotalWages,EstimatedPopulation
0,704,PARC PARQUE,PR,NaN,NaN
1,704,PASEO COSTA DEL SUR,PR,NaN,NaN


In [22]:
df_zip_mean= pd.merge(df_income, df_zip_city_reduced, how='left', left_on='Zip', right_on='Zipcode')

In [25]:
df_zip_mean

,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,mental,name_right,photos,race_right,sources,state_right,threat_level_display,videos,weapon,charged_bool
328,424,Erick Emmanuel Sanchez,2015-04-30,shot and Tasered,metal object,22,M,H,El Paso,TX,...,False,Erick Emmanuel Sanchez,[],H,"[{'source_name': 'El Paso Times.com', 'url': '...",TX,other,[],None,0
964,1113,Chan Leith,2015-12-21,shot,gun,25,M,B,Aurora,CO,...,False,Chan Leith,[],B,"[{'source_name': '9News', 'url': 'http://www.9...",CO,attack,[],None,0


In [49]:
df_zip_mean.head(2)

,zip,median,mean,pop,zipcode,city,state,totalwages,estimatedpopulation
0,1001,"56,663","66,688","16,445",1001,AGAWAM,MA,337735143,14021
1,1002,"49,853","75,063","28,069",1002,AMHERST,MA,415081243,16532


In [50]:
df_zip_mean[['city','state']] = df_zip_mean[['city','state']].apply(lambda x: x.str.lower())
df[['city','state']] = df[['city','state']].apply(lambda x: x.str.lower())                                  

                                   
                                   

### Merge the Above Zip-Mean  Datframe into the original Police Shootings Dataframe 

In [52]:
df_copy = pd.merge(df, df_zip_mean, how='left', on=['city', 'state'])

In [54]:
df = df_copy

In [84]:
df.state.unique()

array(['tx', 'co', 'md', 'ca', 'fl', 'id', 'az', 'ky', 'al', 'sc', 'ak',
       'nj', 'pa', 'wv', 'in', 'wa', 'la', 'wi', 'ks', 'va', 'hi', 'tn',
       'oh', 'ny', 'ok', 'mo', 'nv', 'il', 'mn', 'nm', 'ne', 'ma', 'ga',
       'ct', 'mi', 'or', 'wy', 'dc', 'nc', 'ut', 'ar', 'mt'], dtype=object)

### Get Race of Neighborhood Data

In [60]:
df_race = pickle.load(open('df_race_save.p', 'rb'))

In [242]:
df_race.head(2)

,FIPS,Name,White,Black,Am Indian/Alaskan Native,Asian,Hawaiian/Pac Islander,Other Race,2+Races%,Hispanic,...,Total.2,White.3,Black.3,Am Indian/Alaskan Native.3,Asian.3,Hawaiian/Pac Islander.3,Other Race.3,2+Races%.3,Hispanic.3,Total.3
0,NaN,USA,79%,7%,1%,3%,0%,0%,2%,9%,...,100%,38%,10%,1%,5%,0%,0%,1%,45%,100%
1,10420,"Akron, OH",89%,7%,0%,2%,0%,0%,1%,1%,...,100%,77%,16%,0%,2%,0%,0%,2%,3%,100%


### Racial Composition of Police

In [195]:
import re
df_police_div = pd.read_csv('police-locals.csv')
df_police_div['city'] = df_police_div['city'].apply(lambda x: x.lower())
df_police_div['state']='fpo'
df_police_div['state']= df_police_div['city'].apply(lambda x: x.split(',')[-1].strip())
df_police_div['city'] = df_police_div['city'].apply(lambda x: x.split(',',1)[0].strip())
df_police_div['state'] = df_police_div['state'].apply(lambda x: re.sub('\.','',x).strip())

In [196]:
df_police_div.head()

,city,police_force_size,all,white,non-white,black,hispanic,asian,state
0,new york,32300,0.617957,0.446387,0.764419,0.770891365,0.762860728,0.749235474,new york
1,chicago,12120,0.875000,0.871963,0.877400,0.89740566,0.83982684,0.966666667,chicago
2,los angeles,10100,0.228218,0.152778,0.263848,0.387387387,0.217679558,0.305263158,los angeles
3,washington,9340,0.115632,0.056774,0.157365,0.170189099,0.08988764,0.230769231,washington
4,houston,7700,0.292208,0.173735,0.399258,0.36637931,0.457142857,0.408163265,houston


### The State field is screwed up, make a dictionary and pull the proper value

In [231]:
state_abbrev = {
'chicago': 'il', 'new york' : 'ny', 'washington' : 'wa', 'houston' : 'tx',
'philadelphia': 'pa', 'phoenix' : 'az',  'san diego': 'ca', 'dallas':'tx', 'detroit':'michigan',
'san francisco':'ca', 'san antonio' : 'tx', 'atlanta' : 'ga', 'las vegas' : 'nv', 'baltimore' : 'md',
'boston' : 'ma', 'fla':'fl', 'texas':'tx', 'ohio':'oh', 'cleveland': 'oh', 'arizona':'az', 'tenn':'tn',
'milwaukee': 'wi', 'calif':'ca', 'miami':'fl','denver' : 'co', 'pittsburgh': 'pa' , 'cincinnati':'oh',
'ala':'al', 'kan':'ks', 'minneapolis':'mn', 'ore':'or', 'nev':'nv', 'st louis': 'mo', 'miss': 'ms',
'wis':'wi', 'colo':'co', 'tucson':'az', 'newark' : 'nj','los angeles':'ca', 'charlotte':'nc', 
'rochester':'ny','virginia beach':'va', 'albuquerque': 'nm', 'new orleans':'la','louisville':'ky',
'norfolk':'va', 'arlington':'ny', 'seattle':'wa', 'indianapolis':'in', 'savannah':'ga', 'oklahoma city':'ok',
'jersey city':'nj','baton rouge':'la', 'winston-salem': 'nc', 'jacksonville':'fl','columbus':'oh', 'memphis':'tn',
'san jose':'ca', 'birmingham':'al', 'sacramento':'ca', 'raleigh':'nc', 'tampa':'fl',
 'santa ana':'ca', 'oakland':'ca', 'orlando':'fl', 'kansas city':'ks', 'nashville':'tn','long beach':'ca',
'wichita':'ks', 'fresno':'ca', 'buffalo':'ny', 'portland':'or', 'reno':'nv', 'jackson':'fl',
'riverside': 'ca', 'fort lauderdale':'fl', 'st. louis':'mo', 'albany':'ny', 'colorado springs':'co',
'toledo': 'oh', 'madison':'wi', 'san bernardino':'ca', 'richmond': 'va', 'detroit':'mi', 'texas':'tx'

   }

def state_func(df, state_abbrev=state_abbrev):
    if df['state'] in state_abbrev:
        df['state'] = state_abbrev[df['state']]
    elif df['city'] in state_abbrev:
        df['state'] = state_abbrev[df['city']]
    else:
        df['state'] = 'None'
    return df
        

df_police_div = df_police_div.apply(state_func, axis=1)

# df_police_div[['city','state']][df_police_div.state=='None']

In [233]:
df_police_div.head(2)

,city,police_force_size,all,white,non-white,black,hispanic,asian,state
0,new york,32300,0.617957,0.446387,0.764419,0.770891365,0.762860728,0.749235474,ny
1,chicago,12120,0.875000,0.871963,0.877400,0.89740566,0.83982684,0.966666667,il


In [235]:
df_copy = pd.merge(df, df_police_div, how='left', on=['city','state'])

In [239]:
df = df_copy.copy()

In [241]:
df.head(3)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,zipcode,totalwages,estimatedpopulation,police_force_size,all,white,non-white,black,hispanic,asian
0,424,Erick Emmanuel Sanchez,2015-04-30,shot and Tasered,metal object,22,M,H,el paso,tx,...,79901,91447915,12682,2260,0.85177,0.826446,0.861027,**,0.86102719,**
1,424,Erick Emmanuel Sanchez,2015-04-30,shot and Tasered,metal object,22,M,H,el paso,tx,...,79902,270117955,16056,2260,0.85177,0.826446,0.861027,**,0.86102719,**
2,424,Erick Emmanuel Sanchez,2015-04-30,shot and Tasered,metal object,22,M,H,el paso,tx,...,79903,163746053,14072,2260,0.85177,0.826446,0.861027,**,0.86102719,**


### Get Racial Composition of Neighborhood

In [335]:
df_neighborhoor_race = pickle.load(open('df_race_save.p', 'rb'))
df_neighborhoor_race['State'] = df_neighborhoor_race['Name'].apply(lambda x: x.split(',')[-1].strip())
df_neighborhoor_race['City'] = df_neighborhoor_race['Name'].apply(lambda x: x.split(',')[0].strip())
df_neighborhoor_race['State'] = df_neighborhoor_race['State'].apply(lambda x: re.sub('\.','',x).strip())
df_neighborhoor_race['State'] = df_neighborhoor_race['State'].apply(lambda x: x.lower())
df_neighborhoor_race['City'] = df_neighborhoor_race['City'].apply(lambda x: x.lower())

In [272]:
df_neighborhoor_race.head()

,FIPS,Name,White,Black,Am Indian/Alaskan Native,Asian,Hawaiian/Pac Islander,Other Race,2+Races%,Hispanic,...,Black.3,Am Indian/Alaskan Native.3,Asian.3,Hawaiian/Pac Islander.3,Other Race.3,2+Races%.3,Hispanic.3,Total.3,State,City
0,NaN,USA,79%,7%,1%,3%,0%,0%,2%,9%,...,10%,1%,5%,0%,0%,1%,45%,100%,usa,usa
1,10420,"Akron, OH",89%,7%,0%,2%,0%,0%,1%,1%,...,16%,0%,2%,0%,0%,2%,3%,100%,oh,akron
2,10580,"Albany-Schenectady-Troy, NY",88%,4%,0%,3%,0%,0%,1%,3%,...,15%,0%,3%,0%,1%,2%,7%,100%,ny,albany-schenectady-troy
3,10740,"Albuquerque, NM",53%,2%,4%,2%,0%,0%,2%,37%,...,3%,4%,1%,0%,0%,1%,54%,100%,nm,albuquerque
4,10900,"Allentown-Bethlehem-Easton, PA-NJ",86%,3%,0%,2%,0%,0%,1%,8%,...,8%,0%,2%,0%,0%,2%,32%,100%,pa-nj,allentown-bethlehem-easton


In [336]:
df_nh_race = df_neighborhoor_race[['City','State','White', 'Black', 'Am Indian/Alaskan Native', 'Asian',
       'Hawaiian/Pac Islander', 'Other Race', '2+Races%', 'Hispanic', 'Total',
       'White.1', 'Black.1', 'Am Indian/Alaskan Native.1', 'Asian.1',
       'Hawaiian/Pac Islander.1', 'Other Race.1', '2+Races%.1', 'Hispanic.1',
       'Total.1', 'White.2', 'Black.2', 'Am Indian/Alaskan Native.2',
       'Asian.2', 'Hawaiian/Pac Islander.2', 'Other Race.2', '2+Races%.2',
       'Hispanic.2', 'Total.2', 'White.3', 'Black.3',
       'Am Indian/Alaskan Native.3', 'Asian.3', 'Hawaiian/Pac Islander.3',
       'Other Race.3', '2+Races%.3', 'Hispanic.3', 'Total.3']]


In [337]:
df_nh_race.rename(columns = {
        "White": "W_White",# NEIGHBORHOOD OF AVERAGE WHITE
        "Black": "W_Black",
        "Am Indian/Alaskan Native": "W_Ind_Inuit",
        "Asian": "W_Asian",
        "Hawaiian/Pac Islander": "W_PIslander",
        "Other Race": "W_Other",
        "2+Races%":'W_Mixed',
        "Hispanic":"W_Hispanic",
        "Total": "W_Total",
        "White.1": "B_White",# NEIGHBORHOOD OF AVERAGE BLACK
        "Black.1": "B_Black",
        "Am Indian/Alaskan Native.1": "B_Ind_Inuit",
        "Asian.1": "B_Asian",
        "Hawaiian/Pac Islander.1": "B_PIslander",
        "Other Race.1": "B_Other",
        "2+Races%.1":'B_Mixed',
        "Hispanic.1":"B_Hispanic",
        "Total.1": "B_Total",      
        "White.2": "A_White",# NEIGHBORHOOD OF AVERAGE ASIAN 
        "Black.2": "A_Black",
        "Am Indian/Alaskan Native.2": "A_Ind_Inuit",
        "Asian.2": "A_Asian",
        "Hawaiian/Pac Islander.2": "A_PIslander",
        "Other Race.2": "A_Other",
        "2+Races%.2":'A_Mixed',
        "Hispanic.2":"A_Hispanic",
        "Total.2": "A_Total",      
        "White.3": "H_White",# NEIGHBORHOOD OF AVERAGE Hispanic 
        "Black.3": "H_Black",
        "Am Indian/Alaskan Native.3": "H_Ind_Inuit",
        "Asian.3": "H_Asian",
        "Hawaiian/Pac Islander.3": "H_PIslander",
        "Other Race.3": "H_Other",
        "2+Races%.3":'H_Mixed',
        "Hispanic.3":"H_Hispanic",
        "Total.3":"H_Total"  
        }, inplace = True)


//anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [338]:
df_nh_race['Cities']=df_nh_race.City.apply(lambda x: x.strip().split('-'))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [339]:
df_nh_race.head()

,City,State,W_White,W_Black,W_Ind_Inuit,W_Asian,W_PIslander,W_Other,W_Mixed,W_Hispanic,...,H_White,H_Black,H_Ind_Inuit,H_Asian,H_PIslander,H_Other,H_Mixed,H_Hispanic,H_Total,Cities
0,usa,usa,79%,7%,1%,3%,0%,0%,2%,9%,...,38%,10%,1%,5%,0%,0%,1%,45%,100%,[usa]
1,akron,oh,89%,7%,0%,2%,0%,0%,1%,1%,...,77%,16%,0%,2%,0%,0%,2%,3%,100%,[akron]
2,albany-schenectady-troy,ny,88%,4%,0%,3%,0%,0%,1%,3%,...,71%,15%,0%,3%,0%,1%,2%,7%,100%,"[albany, schenectady, troy]"
3,albuquerque,nm,53%,2%,4%,2%,0%,0%,2%,37%,...,36%,3%,4%,1%,0%,0%,1%,54%,100%,[albuquerque]
4,allentown-bethlehem-easton,pa-nj,86%,3%,0%,2%,0%,0%,1%,8%,...,57%,8%,0%,2%,0%,0%,2%,32%,100%,"[allentown, bethlehem, easton]"


In [348]:
df_nh_race_copy = df_nh_race.copy()

In [351]:
def get_zip(race_df, df= df):  
    cities = race_df['Cities']
    if cities in df['city']: 
        race_df['zipcode'] = df['zipcode'][0] 
        return race_df
#     for i in race_df['Cities']: 
#         #print(cities)
#         Eif not df['city'][df['city'] == i].empty:
#             race_df['zipcode'] = df['zipcode'][0]   


            
#df_police_div = df_police_div.apply(state_func, axis=1)
        
# def state_func(df, state_abbrev=state_abbrev):
#     if df['state'] in state_abbrev:
#         df['state'] = state_abbrev[df['state']]
#     elif df['city'] in state_abbrev:
#         df['state'] = state_abbrev[df['city']]
#     else:
#         df['state'] = 'None'
#     return df
                    
    
    #get_zip(['el paso', 'austin', 'houston'], df) 



#pd.notnull(df.zipcode[df.city=='el paso'][0])
#df_nh_race.City.value_counts()

df_nh_race_copy = df_nh_race.apply(get_zip, axis=1)


TypeError: ("unhashable type: 'list'", 'occurred at index 0')

In [347]:
df_nh_race_copy.columns

Index(['A_Asian', 'A_Black', 'A_Hispanic', 'A_Ind_Inuit', 'A_Mixed', 'A_Other',
       'A_PIslander', 'A_Total', 'A_White', 'B_Asian', 'B_Black', 'B_Hispanic',
       'B_Ind_Inuit', 'B_Mixed', 'B_Other', 'B_PIslander', 'B_Total',
       'B_White', 'Cities', 'City', 'H_Asian', 'H_Black', 'H_Hispanic',
       'H_Ind_Inuit', 'H_Mixed', 'H_Other', 'H_PIslander', 'H_Total',
       'H_White', 'State', 'W_Asian', 'W_Black', 'W_Hispanic', 'W_Ind_Inuit',
       'W_Mixed', 'W_Other', 'W_PIslander', 'W_Total', 'W_White', 'zipcode'],
      dtype='object')

### Pickle Save

In [243]:
 pickle.dump(df, open( "df_big_table_save.p", "wb" ) )